Load stacked edge data, and write basic viz functions to plot edges etc.

In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt

import opinf

In [2]:
import re
import os

In [3]:
from multiprocessing import Pool
from itertools import repeat

In [4]:
plt.rc("axes.spines", right=True, top=True)
plt.rc("figure", dpi=300, 
       figsize=(9, 3)
      )
plt.rc("font", family="serif")
plt.rc("legend", edgecolor="none", frameon=True)
# plt.rc("text", usetex=True)

In [5]:
def load_edge_data(eventID):
    edge_data = np.load("./edge_data/CR{}_stacked_edge.npy".format(eventID))
    simID_data = np.load("./edge_data/CR{}_SimID4edge.npy".format(eventID))
    
    all_sims = []
    
    for s in simID_data:
        mm = re.search("run\d\d\d_AWSoM2T", s)
        all_sims.append(int(mm.group().split("run")[1].split("_")[0]))
    
    return edge_data, np.array(all_sims)

In [6]:
ed_2161, sd_2161 = load_edge_data(2161)
ed_2192, sd_2192 = load_edge_data(2192)
ed_2154, sd_2154 = load_edge_data(2154)

In [7]:
nTimes = ed_2161.shape[0]
nTimes

90

In [8]:
nTheta_2161 = ed_2161.shape[1]
nTheta_2192 = ed_2192.shape[1]
nTheta_2154 = ed_2154.shape[1]
nTheta_2161, nTheta_2192, nTheta_2154

(200, 185, 400)

In [9]:
nSims_2161 = ed_2161.shape[2]
nSims_2192 = ed_2192.shape[2]
nSims_2154 = ed_2154.shape[2]
nSims_2161, nSims_2192, nSims_2154

(278, 231, 141)

Workflow to filter data before it reaches boundary i.e. look for values going along indices above 120 , pick the minimum row index as cutoff for entire range of angles, and set that as the time up to which we have training and or testing data.

In [10]:
R_inner = 4.05
R_outer = 23.6

In [11]:
height_flattened=128
np.arange(0, height_flattened, step=21)

array([  0,  21,  42,  63,  84, 105, 126])

In [12]:
hts_pixel = np.linspace(0, height_flattened, height_flattened + 1)
hts_pixel

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128.])

In [13]:
# is this correct?
radii = ((hts_pixel)/height_flattened)*(R_outer - R_inner) + R_inner
radii

array([ 4.05      ,  4.20273437,  4.35546875,  4.50820312,  4.6609375 ,
        4.81367187,  4.96640625,  5.11914062,  5.271875  ,  5.42460938,
        5.57734375,  5.73007813,  5.8828125 ,  6.03554687,  6.18828125,
        6.34101562,  6.49375   ,  6.64648438,  6.79921875,  6.95195312,
        7.1046875 ,  7.25742188,  7.41015625,  7.56289062,  7.715625  ,
        7.86835937,  8.02109375,  8.17382812,  8.3265625 ,  8.47929687,
        8.63203125,  8.78476563,  8.9375    ,  9.09023437,  9.24296875,
        9.39570313,  9.5484375 ,  9.70117188,  9.85390625, 10.00664062,
       10.159375  , 10.31210938, 10.46484375, 10.61757812, 10.7703125 ,
       10.92304688, 11.07578125, 11.22851562, 11.38125   , 11.53398437,
       11.68671875, 11.83945313, 11.9921875 , 12.14492188, 12.29765625,
       12.45039063, 12.603125  , 12.75585938, 12.90859375, 13.06132812,
       13.2140625 , 13.36679687, 13.51953125, 13.67226563, 13.825     ,
       13.97773438, 14.13046875, 14.28320312, 14.4359375 , 14.58

In [14]:
ed_2161_scaled = (ed_2161/height_flattened)*(R_outer - R_inner) + R_inner
ed_2192_scaled = (ed_2192/height_flattened)*(R_outer - R_inner) + R_inner
ed_2154_scaled = (ed_2192/height_flattened)*(R_outer - R_inner) + R_inner

Adapted function from `plot_heat_data` in previous notebooks.

In plotting we need to convert axes indices to actual theta and R values (also control the ticks) - follow basic outline of `Kmeans_CR2154_SignedDist` from Hongfan.

**Calculation for angles**

In [15]:
theta_s_2161, theta_e_2161 = np.linspace(0, 360, 512)[160] + 1.2 * 180 - 360, np.linspace(0, 360, 512)[359] + 1.2 * 180 - 360
print("Range of angles for CR2161: {} {}".format(theta_s_2161, theta_e_2161))

theta_s_2192, theta_e_2192 = np.linspace(0, 360, 512)[90] + 1.2 * 180 - 360, np.linspace(0, 360, 512)[274] + 1.2 * 180 - 360
print("Range of angles for CR2161: {} {}".format(theta_s_2192, theta_e_2192))

theta_s_2154, theta_e_2154 = np.linspace(0, 360, 512)[50] + 1.2 * 180 - 360, np.linspace(0, 360, 512)[449] + 1.2 * 180 - 360
print("Range of angles for CR2161: {} {}".format(theta_s_2154, theta_e_2154))

Range of angles for CR2161: -31.279843444227026 108.91585127201563
Range of angles for CR2161: -80.59491193737767 49.03326810176122
Range of angles for CR2161: -108.77495107632095 172.32093933463796


~~Can we also make a Cartesian plot? easier to see all edges progressing outward (with overlaid radius of occulting disk) and get a better sense of the shape.~~

Save for all sims and events.

In [22]:
actualTimes = np.arange(2, 182, 2)

In [23]:
edge_save_dir = "./edge_img"

In [44]:
ed_2192.max()

136.27816512833152

In [32]:
def plot_edge_data(Z, titleSim, eventID=2161, plotAll=False, ax=None, theta=np.linspace(-32, 107, 90)):
    """Visualize edges data in space and time both in polar and original Cartesian coordinates."""
    if ax is None:
        _, ax = plt.subplots(1, 1)

    # Plot a few snapshots over the spatial domain.
    if plotAll:
        sample_columns = [i for i in range(Z.shape[1])]
    else:
        sample_columns = [0, 2, 4, 8, 12, 20, 40, 50, 60, 70, 74, 80, 84, 88, 89]
        
    sample_times = actualTimes[sample_columns]
    color = iter(plt.cm.viridis(np.linspace(0, 1, len(sample_columns))))
    while sample_columns[-1] > Z.shape[1]:
        sample_columns.pop()
    for i, j in enumerate(sample_columns):
        ax.plot(theta, Z[:, j], color=next(color), label=fr"$u(t_{{{sample_times[i]}}})$")

    ax.set_xlabel(r"$theta (deg)$")
    ax.set_ylabel(r"$r_{edge}(t)$")
    
    ax.set_xlim(theta[0], theta[-1])
    ax.set_yticks(np.arange(0, height_flattened, step=21), labels=['4R', '7R', '10R', '13R', '16R', '19R', '24R'])
    ax.legend(loc=(1.05, .05))
    ax.set_title("Edge data Sim {:03d}".format(titleSim))
    
    plt.savefig(os.path.join(edge_save_dir, "CR{}".format(eventID), "run_{:03d}.png".format(titleSim)),
               bbox_inches="tight", pad_inches=0)
    
    print("Saved image for Event {} Sim {:03d}".format(eventID, titleSim))
    
    plt.close()

In [36]:
def save_event_edge_data(eventID, plotAll=False, ax=None, theta_start=160, theta_end=360):
    # call data simwise, get sim labels
    edge_data, sim_data = load_edge_data(eventID)
    edge_data = (edge_data/height_flattened)*(R_outer - R_inner) + R_inner
    # make call to plotting function (also saves image) - MultiProcessing?
    nSims = edge_data.shape[2]
    
    theta_range = np.linspace(0, 360, 512)[theta_start:theta_end] + 1.2 * 180 - 360
    for i in range(nSims):
        plot_edge_data(edge_data[:, :, i].T, sim_data[i], eventID=eventID, 
                       plotAll=False, theta=theta_range)        

Are our ylims getting set correctly in the figure?

There is definitely some error in the plotting. The edge at the last timestep is below the starting edge in many cases which is definitely wrong.

In [37]:
plt.ioff()
# plt.ion()

In [39]:
# # commented out call for saving edge data
# save_event_edge_data(2161, plotAll=False, ax=None, theta_start=160, theta_end=360)
# save_event_edge_data(2192, plotAll=False, ax=None, theta_start=90, theta_end=275)
# save_event_edge_data(2154, plotAll=False, ax=None, theta_start=50, theta_end=450)

In [ ]:
# POLAR AXES PLOT
# fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
# # r = np.linspace(4.05, 23.6, height_flattened + 1)
# theta = np.linspace(-32, 107, 90) * np.pi / 180
# ax.plot(theta, ed_2161_scaled[:, :, 0].T[25, :])
# ax.plot(theta, ed_2161_scaled[:, :, 0].T[40, :])
# ax.plot(theta, ed_2161_scaled[:, :, 0].T[50, :])

In [ ]:
# plot_edge_data(edge_data_cutoff.T, "Edge data Sim 31", plotAll=False)

**Scratch Begin**

In [ ]:
edge_data_31 = ed_2161[:, :, 0]

In [ ]:
tIdx = min(np.unique(np.where(edge_data_31 >= 120)[0]))
tIdx

In [ ]:
tCutoff = actualTimes[tIdx]
tCutoff

In [ ]:
edge_data_cutoff = edge_data_31[:tIdx, :]
edge_data_cutoff.shape

In [ ]:
sample_columns = [0, 2, 4, 8, 12, 20, 40, 50, 60, 70]
sample_times = actualTimes[sample_columns]
sample_times

In [ ]:
basis = opinf.pre.PODBasis().fit(edge_data_cutoff.T, residual_energy=1e-8)
print(basis)

# Check the decay of the singular values.
basis.plot_svdval_decay()
plt.xlim(0, 25)

In [ ]:
plt.plot(basis.svdvals/basis.svdvals[1])
plt.xlim(0, 10)

**Scratch End**